In [0]:
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [25]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
# HELPER
# INIT WEIGHTS
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [0]:
# INIT BIAS
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [0]:
# CONV2D
def conv2d(x,w):
    # x --> [batch,H,W,Channels]
    # W --> [filter H,filter W,Channels IN, Channels OUT]
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')

In [0]:
# POOLING
def max_pool_2by2(x):
    # x --> [batch,h,w,c]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [0]:
# Convolutional Layer
def convolutional_layer(input_x,shape):
    w = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,w)+b)

In [0]:
# NORMAL (FULLY CONNECTED)
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    w = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,w)+ b

In [0]:
# Placeholders
x = tf.placeholder(tf.float32,shape=[None,784])

In [0]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [0]:
# Layers
x_image = tf.reshape(x,[-1,28,28,1])

In [0]:
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [0]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
convo_2_polling = max_pool_2by2(convo_2)

In [0]:
convo_2_flat = tf.reshape(convo_2_polling,[-1,7*7*64])
fully_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [0]:
# DROPOUT
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(fully_layer_one,rate=hold_prob)

In [0]:
y_pred = normal_full_layer(full_one_dropout,10)

In [0]:
# LOSS FUNCTION
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true,logits=y_pred))

In [0]:
# OPTIMIZER
optimizer = tf.train.AdadeltaOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [0]:
init = tf.global_variables_initializer()

In [44]:
steps = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x,batch_y = mnist.train.next_batch(50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        if i%100 == 0:
            print("ON STEP: {}".format(i))
            print("ACCURACY: ")
            matches = tf.equal(tf.arg_max(y_pred,1),tf.arg_max(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

ON STEP: 0
ACCURACY: 
0.098


ON STEP: 100
ACCURACY: 
0.098


ON STEP: 200
ACCURACY: 
0.098


ON STEP: 300
ACCURACY: 
0.098


ON STEP: 400
ACCURACY: 
0.098


ON STEP: 500
ACCURACY: 
0.098


ON STEP: 600
ACCURACY: 
0.098


ON STEP: 700
ACCURACY: 
0.098


ON STEP: 800
ACCURACY: 
0.098


ON STEP: 900
ACCURACY: 
0.098


ON STEP: 1000
ACCURACY: 
0.098


ON STEP: 1100
ACCURACY: 
0.098


ON STEP: 1200
ACCURACY: 
0.098


ON STEP: 1300
ACCURACY: 
0.098


ON STEP: 1400
ACCURACY: 
0.098


ON STEP: 1500
ACCURACY: 
0.098


ON STEP: 1600
ACCURACY: 
0.098


ON STEP: 1700
ACCURACY: 
0.098


ON STEP: 1800
ACCURACY: 
0.098


ON STEP: 1900
ACCURACY: 
0.098


ON STEP: 2000
ACCURACY: 
0.098


ON STEP: 2100
ACCURACY: 
0.098


ON STEP: 2200
ACCURACY: 
0.098


ON STEP: 2300
ACCURACY: 
0.098


ON STEP: 2400
ACCURACY: 
0.098


ON STEP: 2500
ACCURACY: 
0.098


ON STEP: 2600
ACCURACY: 
0.098


ON STEP: 2700
ACCURACY: 
0.098


ON STEP: 2800
ACCURACY: 
0.098


ON STEP: 2900
ACCURACY: 
0.098


ON STEP: 3000
ACCURACY